#### FUTURE JOBS
##### GET THE LATEST DATA CHECKING MISSING VALUES

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

from tqdm import tqdm

import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import gc

import warnings
warnings.filterwarnings('ignore')

print(f'DONE.....')

In [ ]:
# 크롤링 전 세팅
# driver 설정 위치 세팅
chrome_driver_path = r'C:/Users/HYUN/Documents/py_data/chromedriver.exe'

# chrome driver options
options = Options()
options.add_argument('--start-maximized')
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument('--disable-gpu')
# options.add_argument("--headless")
 
def infinite_loop():
    #스크롤 내리기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollBy(0, document.documentElement.scrollHeight);")
        time.sleep(1.0)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_page_height == last_page_height:
            time.sleep(1.0)
            if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

print(f'Setting DONE...')

### GET ONE BY ONE COIN DATA

In [ ]:
# 1. coin market list down
# Total around 9400

# page structure
# https://coinmarketcap.com/currencies/bitcoin/historical-data/
base_url = 'https://coinmarketcap.com/currencies/'
coin_name = 'ssv-network'
end_url = '/historical-data/'

target_url = base_url + coin_name + end_url

In [ ]:
# OPEN THE CHROME

driver = Chrome(executable_path = chrome_driver_path, options=options)    
wait = WebDriverWait(driver, 300)
driver.get(target_url) # 영상 url
time.sleep(1.5)


In [ ]:
# 더보기 버튼 클릭

# 최초 1회 마지막 페이지로 이동
driver.execute_script("window.scrollBy(0, document.documentElement.scrollHeight);")
time.sleep(1.5)

# 맨 마지막 페이지 가도 버튼은 그대로 살아있음
# 대신에 페이지 길이는 사라지지 않음
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
new_page_height = 0

while last_page_height != new_page_height:
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div/div[1]/p[1]').click()
    time.sleep(0.5)

    infinite_loop()
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

print(f'END of PAGE....')

In [ ]:
# data url
# table : driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div/div[1]/div[2]/table/tbody')
# each row : .find_elements_by_tag_name('tr')
# data : .find_elements_by_tag_name('td')

name_lst = list()
date_lst = list()
open_lst = list()
high_lst = list()
low_lst = list()
close_lst = list()
volume_lst = list()
cap_lst = list()

for data_row in tqdm(driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div/div[1]/div[2]/table/tbody').find_elements_by_tag_name('tr')):

    # 1. Date list
    date_temp = data_row.find_elements_by_tag_name('td')[0].text
    date_lst.append(date_temp)

    # 2. open list
    open_temp = data_row.find_elements_by_tag_name('td')[1].text
    open_lst.append(open_temp)

    # 3. high list 
    high_temp = data_row.find_elements_by_tag_name('td')[2].text
    high_lst.append(high_temp)

    # 4. low list
    low_temp = data_row.find_elements_by_tag_name('td')[3].text
    low_lst.append(low_temp)

    # 5. close list
    close_temp = data_row.find_elements_by_tag_name('td')[4].text
    close_lst.append(close_temp)

    # 6. volume list
    volume_temp = data_row.find_elements_by_tag_name('td')[5].text
    volume_lst.append(volume_temp)

    # 7. market cap list
    cap_temp = data_row.find_elements_by_tag_name('td')[6].text
    cap_lst.append(cap_temp)

# 0. Coin name
name_lst = [coin_name for i in range(len(date_lst))]

print(f'{coin_name} crawled....')

In [ ]:
# Save the result

coin_df = pd.DataFrame(data=[name_lst, date_lst, open_lst, high_lst, low_lst, close_lst, volume_lst, cap_lst]).T

coin_df.rename(columns={
    0 : 'Name',
    1 : 'Date', 
    2 : 'Open_price', 
    3 : 'High_price', 
    4 : 'Low_price', 
    5 : 'Close_price', 
    6: 'Volume', 
    7 : 'Market_cap'
    }, inplace=True)

coin_df.head()

In [ ]:
driver.quit()

### SAVE TOTAL COIN DATA

In [119]:
# Start crawling data...

coin_df = pd.read_csv('./0. coin list/coin_list_230319.csv', index_col=0)

coin_name_lst = coin_df['NAME'].tolist()
print(f'csv converted......')

csv converted......


In [ ]:
# 1. coin market list down
# Total around 9000

# page structure
# https://coinmarketcap.com/currencies/bitcoin/historical-data/

final_df = pd.DataFrame()

for coin_name in tqdm(coin_name_lst):
    print(f'{coin_name} started....\n')
    base_url = 'https://coinmarketcap.com/currencies/'
    end_url = '/historical-data/'

    target_url = base_url + coin_name + end_url

    # OPEN THE CHROME
    driver = Chrome(executable_path = chrome_driver_path, options=options)    
    wait = WebDriverWait(driver, 300)
    driver.get(target_url) # 영상 url
    time.sleep(1.5)

    # 더보기 버튼 클릭

    # 최초 1회 마지막 페이지로 이동
    driver.execute_script("window.scrollBy(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    # 맨 마지막 페이지 가도 버튼은 그대로 살아있음
    # 대신에 페이지 길이는 사라지지 않음
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    new_page_height = 0

    while last_page_height != new_page_height:
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div/div[1]/p[1]').click()
        time.sleep(0.1)

        infinite_loop()
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # data url
    # table : driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div/div[1]/div[2]/table/tbody')
    # each row : .find_elements_by_tag_name('tr')
    # data : .find_elements_by_tag_name('td')

    name_lst = list()
    date_lst = list()
    open_lst = list()
    high_lst = list()
    low_lst = list()
    close_lst = list()
    volume_lst = list()
    cap_lst = list()

    for data_row in tqdm(driver.find_element_by_xpath('//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div/div/div[1]/div[2]/table/tbody').find_elements_by_tag_name('tr')):

        # 1. Date list
        date_temp = data_row.find_elements_by_tag_name('td')[0].text
        date_lst.append(date_temp)

        # 2. open list
        open_temp = data_row.find_elements_by_tag_name('td')[1].text
        open_lst.append(open_temp)

        # 3. high list 
        high_temp = data_row.find_elements_by_tag_name('td')[2].text
        high_lst.append(high_temp)

        # 4. low list
        low_temp = data_row.find_elements_by_tag_name('td')[3].text
        low_lst.append(low_temp)

        # 5. close list
        close_temp = data_row.find_elements_by_tag_name('td')[4].text
        close_lst.append(close_temp)

        # 6. volume list
        volume_temp = data_row.find_elements_by_tag_name('td')[5].text
        volume_lst.append(volume_temp)

        # 7. market cap list
        cap_temp = data_row.find_elements_by_tag_name('td')[6].text
        cap_lst.append(cap_temp)

    # 0. Coin name
    name_lst = [coin_name for i in range(len(date_lst))]

    temp_df = pd.DataFrame(data=[name_lst, date_lst, open_lst, high_lst, low_lst, close_lst, volume_lst, cap_lst]).T

    final_df = pd.concat([final_df, coin_df], axis=0, ignore_index=True)

    del temp_df
    del name_lst, date_lst, open_lst, high_lst, low_lst, close_lst, volume_lst, cap_lst

    gc.collect()

    print(f'{coin_name} crawled....\n')